In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import cv2

Y_START,Y_END=60,135
IMG_SIZE=(200,66)
BATCH = 32




### Data Augmentation

In [ ]:
def augment_image(
    image, 
    steering_angle, 
    flip_prob=0.5, 
    brightness_prob=0.5, 
    zoom_prob=0.5, 
    pan_prob=0.5, 
    rotate_prob=0.5,
    max_brightness_delta=0.25,   # ~±25% brightness
    max_zoom=0.2,                # up to 20% zoom-in
    max_translation=0.1,         # up to 10% shift in height/width
    max_rotation=0.05            # ~±5% of 2π (~±9°)
):
    """
    Randomly augments an image and adjusts steering angle accordingly.
    
    Args:
        image: 3D tf.Tensor [H, W, C], dtype uint8 or float32.
        steering_angle: float scalar (Python float or 0-D tf.Tensor).
        *_prob: probability of applying each augmentation (0..1).
        max_*: magnitude controls for each operation.
    Returns:
        aug_image: augmented image, float32 in [0,1], shape [H, W, C]
        aug_angle: adjusted steering angle (flipped if image flipped)
    """
    # Ensure float32 in [0,1]
    img = tf.image.convert_image_dtype(image, dtype=tf.float32)
    angle = tf.cast(steering_angle, tf.float32)

    H = tf.shape(img)[0]
    W = tf.shape(img)[1]
    
 # -------- Flip (horizontal) --------
    # If applied, invert steering angle
    do_flip = tf.less(tf.random.uniform([]), flip_prob)
    img = tf.cond(
        do_flip,
        lambda: tf.image.flip_left_right(img),
        lambda: img
    )
    angle = tf.cond(do_flip, lambda: -angle, lambda: angle)

    # -------- Brightness --------
    def brightness(x):
        return tf.clip_by_value(
            tf.image.random_brightness(x, max_delta=max_brightness_delta),
            0.0, 1.0
        )
    img = tf.cond(tf.less(tf.random.uniform([]), brightness_prob),
                  lambda: brightness(img),
                  lambda: img)

    # -------- Zoom (crop center, then resize back) --------
    # Zoom-in only (keeps size by resizing back); simpler & robust
    def zoom(x):
        zoom_factor = 1.0 - tf.random.uniform([], 0.0, max_zoom)  # e.g., 0.8..1.0
        new_h = tf.cast(tf.round(tf.cast(H, tf.float32) * zoom_factor), tf.int32)
        new_w = tf.cast(tf.round(tf.cast(W, tf.float32) * zoom_factor), tf.int32)
        # center-crop to [new_h, new_w]
        offset_h = (H - new_h) // 2
        offset_w = (W - new_w) // 2
        cropped = tf.image.crop_to_bounding_box(x, offset_h, offset_w, new_h, new_w)
        return tf.image.resize(cropped, (H, W), method="bilinear")
    img = tf.cond(tf.less(tf.random.uniform([]), zoom_prob),
                  lambda: zoom(img),
                  lambda: img)

    # -------- Pan (translation) --------
    # Implemented via pad + random crop to original size
    def panning(x):
        # pad by up to max_translation * size, then crop a different window
        pad_h = tf.cast(tf.round(tf.cast(H, tf.float32) * max_translation), tf.int32)
        pad_w = tf.cast(tf.round(tf.cast(W, tf.float32) * max_translation), tf.int32)
        padded = tf.pad(x, [[pad_h, pad_h], [pad_w, pad_w], [0, 0]], mode="REFLECT")
        # pick a random top-left for crop
        max_off_h = 2 * pad_h
        max_off_w = 2 * pad_w
        off_h = tf.random.uniform([], 0, max_off_h + 1, dtype=tf.int32)
        off_w = tf.random.uniform([], 0, max_off_w + 1, dtype=tf.int32)
        return tf.image.crop_to_bounding_box(padded, off_h, off_w, H, W)
    img = tf.cond(tf.less(tf.random.uniform([]), pan_prob),
                  lambda: panning(img),
                  lambda: img)

    # -------- Rotation --------
    # Use Keras preprocessing for rotation (works per-sample)
    def rotate(x):
        layer = tf.keras.layers.RandomRotation(factor=max_rotation)
        # layer expects a batch; set training=True to ensure effect
        x = layer(tf.expand_dims(x, 0), training=True)
        return tf.squeeze(x, 0)
    img = tf.cond(tf.less(tf.random.uniform([]), rotate_prob),
                  lambda: rotate(img),
                  lambda: img)

    return img, angle

### Data preprocessing


In [ ]:
def preprocess_image(img_bgr):
    
    # Crop the image to get the road
    cropped = img_bgr[60:136, :, :]
    
    # Convert BGR to YUV
    yuv = cv2.cvtColor(cropped, cv2.COLOR_BGR2YUV)

    # Gaussian blur
    yuv = cv2.GaussianBlur(yuv, (3, 3), 0)

    # Resize
    resized = cv2.resize(yuv, (200, 66), interpolation=cv2.INTER_AREA)

    # Normalize pixel values to range[0,1]
    processed = resized.astype(np.float32) / 255.0

    return processed

### Model 